In [2]:
from pathlib import Path
from ragdaemon.daemon import Daemon

annotators = {
    "hierarchy": {},
    "chunker": {},
    # "call_graph": {},
    # "summarizer": {},
    "diff": {},
}

# sample_dir = Path.home() / "butler" / "benchmark" / "swe_benchmarks" / "repos" / "sympy__sympy-23262"
sample_dir = Path.cwd()
daemon = Daemon(
    cwd=sample_dir, 
    annotators=annotators, 
    verbose=2
)
await daemon.update()

Initialized empty graph.
Initializing annotators: ['hierarchy', 'chunker', 'diff']...


Chunking files...: 100%|██████████| 55/55 [00:02<00:00, 20.87it/s]

Saved updated graph to /Users/granthawkins/.mentat/ragdaemon/ragdaemon-ragdaemon.json


In [24]:
ans = await daemon.answer("Show me the diff from c556afd1f. Use subprocess.")

Script: import subprocess

# Retrieve the diff for the provided commit hash
commit_hash = "c556afd1f"
diff = subprocess.run(["git", "diff", commit_hash], capture_output=True, text=True)

print(diff.stdout)
--------------------------------------------------------------------------------
Output: diff --git a/pyproject.toml b/pyproject.toml
index fd4a34f..33dd123 100644
--- a/pyproject.toml
+++ b/pyproject.toml
@@ -7,7 +7,7 @@ packages=["ragdaemon"]
 
 [project]
 name = "ragdaemon"
-version = "0.7.0"
+version = "0.7.5"
 description = "Generate and render a call graph for a Python project."
 readme = "README.md"
 dependencies = [
diff --git a/ragdaemon/__init__.py b/ragdaemon/__init__.py
index 49e0fc1..ab55bb1 100644
--- a/ragdaemon/__init__.py
+++ b/ragdaemon/__init__.py
@@ -1 +1 @@
-__version__ = "0.7.0"
+__version__ = "0.7.5"
diff --git a/ragdaemon/annotators/hierarchy.py b/ragdaemon/annotators/hierarchy.py
index b32cb22..661953c 100644
--- a/ragdaemon/annotators/hierarchy.py
+++ b/ragd

In [25]:
print(ans)


The diff for commit `c556afd1f` includes the following changes:

1. In `pyproject.toml`:
   - Version updated from `0.7.0` to `0.7.5`.
2. In `ragdaemon/__init__.py`:
   - Version updated from `0.7.0` to `0.7.5`.
3. In `ragdaemon/annotators/hierarchy.py`:
   - Modification to include `ignore_patterns` when calling `get_document`.
   - Changed checksumming logic to use `checksums.get` instead of `checksums`.
4. In `ragdaemon/annotators/summarizer.py`:
   - Reformatted a block of code for better readability.
5. Added a new file `ragdaemon/cerebrus.py` with substantial content related to parsing scripts, a `Printer` class, and an async function `cerebrus`.
6. In `ragdaemon/context.py`:
   - Added a new method `to_ids` in `ContextBuilder` class to return a list of ids.
7. In `ragdaemon/daemon.py`:
   - Imported and integrated `cerebrus` in the async `answer` method.
8. In `ragdaemon/locate.py`:
   - Added a new validation function `validate`.
   - Used `partial` from `functools`.
   - Adju

In [3]:
from pathlib import Path
from ragdaemon.daemon import Daemon

cwd = Path.home() / "Desktop" / "testrepo"
daemon = Daemon(cwd)
await daemon.update()
print([n for n in daemon.graph.nodes])

['file1.py', 'ROOT', 'file1.py:BASE', 'file1.py:hello', 'DEFAULT']


In [4]:
# Create a new file, file2, with hello world
file2 = cwd / "file2"
file2.write_text("hello world")
# Stage it with git
# import subprocess
# subprocess.run(["git", "add", "file2"], cwd=cwd)
# subprocess.run(["git", "commit", "-m", "add file2"], cwd=cwd)

await daemon.update()
print([n for n in daemon.graph.nodes])


['file2', 'file1.py', 'ROOT', 'file1.py:BASE', 'file1.py:hello', 'DEFAULT']


In [1]:
from ragdaemon.get_paths import get_paths_for_directory
from ragdaemon.annotators.hierarchy import files_checksum


daemon.pipeline['hierarchy'].is_complete(daemon.graph, None)
print(daemon.graph.graph.get("files_checksum"))
print(files_checksum(cwd))

for path in get_paths_for_directory(cwd):
    print(str((cwd / path).stat().st_mtime))

NameError: name 'daemon' is not defined